<a href="https://colab.research.google.com/github/Nipun142/Transaction-Analysis/blob/main/Transaction_data_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import imaplib
import email
from email.header import decode_header
import os

l=[]
EMAIL = # Enter your own mail id
PASSWORD = # Enter your own app password
MAIL_SERVER = #mail server ex. - "imap.gmail.com"


# Define the date range
START_DATE = "01-Aug-2024"
END_DATE = "01-Sep-2024"

def connect_to_mail():

    try:
        mail = imaplib.IMAP4_SSL(MAIL_SERVER)
        mail.login(EMAIL, PASSWORD)
        return mail
    except imaplib.IMAP4.error as e:
        print(f"Error connecting to mail server: {e}")
        return None

def fetch_emails(mail):
    try:
        mail.select("inbox")

        # Validate date strings
        if not (START_DATE and END_DATE):
            print("Date range is not properly defined.")
            return []

        # Construct search query
        search_criteria = f'(FROM "alerts@axisbank.com" SUBJECT "Debit notification from Axis Bank" SINCE "{START_DATE}" BEFORE "{END_DATE}")'


        status, message_numbers = mail.search(None, search_criteria)
        if status != "OK":
            print("Failed to search emails.")
            return []

        message_numbers = message_numbers[0].split()
        if not message_numbers:
            print("No messages found.")
            return []

        return message_numbers
    except Exception as e:
        print(f"Error fetching emails: {e}")
        return []

def extract_relevant_info(body):
    return body.strip()

def process_email(mail, num):
    try:
        status, msg_data = mail.fetch(num, '(RFC822)')
        if status != "OK":
            print(f"Failed to fetch email ID {num}.")
            return

        msg = email.message_from_bytes(msg_data[0][1])

        # Decode email subject
        subject, encoding = decode_header(msg["Subject"])[0]
        if isinstance(subject, bytes):
            subject = subject.decode(encoding if encoding else "utf-8")
        else:
            subject = subject or "No Subject"

        # Decode email sender
        from_ = msg.get("From") or "Unknown Sender"


        # Process email body
        if msg.is_multipart():
            for part in msg.walk():
                content_type = part.get_content_type()
                content_disposition = str(part.get("Content-Disposition"))

                if "attachment" not in content_disposition:
                    body = part.get_payload(decode=True)
                    if body:
                        body = body.decode(errors='ignore')
                        formatted_body = extract_relevant_info(body)
                        # print(f"Formatted Body: {formatted_body}")
                        l.append(formatted_body)


    except Exception as e:
        print(f"Error processing email ID {num}: {e}")

def main():
    if not (EMAIL and PASSWORD):
        print("Email or Password is not set.")
        return

    mail = connect_to_mail()
    if not mail:
        return

    message_numbers = fetch_emails(mail)
    if not message_numbers:
        return

    for num in message_numbers:
        process_email(mail, num)

    mail.logout()

if __name__ == "__main__":
    main()




transactions = []
df_list=[]
for i in range(0,len(l),2):
  try:
    transactions.append(l[i][:l[i].index(" For any concerns")])
  except:
    continue

for i in range(0,len(transactions)):
  date =  transactions[i][:10]
  amount = float(transactions[i][transactions[i].index("INR "):transactions[i].index(" has")][4:])
  time = transactions[i][transactions[i].index("at "):transactions[i].index(" IST")][3:]
  payment_type = transactions[i][transactions[i].index("Info- "):][6:].split("/")[0]
  merchant_or_personal = transactions[i][transactions[i].index("Info- "):][6:].split("/")[1]
  id = transactions[i][transactions[i].index("Info- "):][6:].split("/")[2]
  name = transactions[i][transactions[i].index("Info- "):][6:].split("/")[3]
  if name[-1]=='.':
    name=name[:-1]
  df_list.append([date,time,id,name,payment_type,amount,merchant_or_personal])





import pandas as pd
cols = ["date","time","id","name","payment_type","amount","merchant_or_personal"]
df = pd.DataFrame(df_list,columns=cols)

df['datetime_str'] = df['date'] + ' ' + df['time']

# Convert the combined string to datetime
df['datetime'] = pd.to_datetime(df['datetime_str'], format='%d-%m-%Y %H:%M:%S')

# Drop the intermediate column if not needed
df = df.drop(columns=['datetime_str','date','time'])


In [5]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 16.8 MB/s eta 0:00:00


In [ ]:
from pymongo import MongoClient

mongo_uri = # your uri string
client = MongoClient(mongo_uri, tls=True, tlsAllowInvalidCertificates=True)  # Allow invalid certificates

# Rest of your code
db = client['your_databse']          #enter your own databse name
collection = db['your_collection']    #enter your own collection name
# Convert DataFrame to dictionary
data_dict = df.to_dict("records")

# Insert data into MongoDB
collection.insert_many(data_dict)

print("Data uploaded successfully to MongoDB!")

Data uploaded successfully to MongoDB!


In [1]:
print("xsa")

xsa
